In [6]:
days = 20*5

DemandB1 = [2600 2900 2500 3000 2400 0]

Ffactor = 300;
Gfactor = 30;
C31factor = 60;
B11factor = 60;

InvB1 = zeros(days+50,1)
InvF = zeros(days+50,1)
InvG = zeros(days+50,1)
InvC31 = zeros(days+50,1)
InvB11 = zeros(days+50,1)

InvF[1:end] = 100;
InvG[1:end] = 10;
InvB11[1:end] = 5;

cycleB1 = 736


t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvB1[t:end] -= DemandB1[demandMonth]
        demandMonth += 1
    end
    if InvB1[t] > DemandB1[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end
    Fleft = (DemandB1[demandMonth] - InvB1[t]) * Ffactor
    Gleft = (DemandB1[demandMonth] - InvB1[t]) * Gfactor
    B11left = (DemandB1[demandMonth] - InvB1[t]) * B11factor
    C31left = (DemandB1[demandMonth] - InvB1[t]) * C31factor
    if ((InvF[t] < Fleft) || (InvG[t] < Gleft) || (InvC31[t] < C31left) ||(InvB11[t] < B11left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvF[tLast:t] = InvF[tLast]
        InvG[tLast:t] = InvG[tLast]
        InvB11[tLast:t] = InvB11[tLast]
        InvC31[tLast:t] = InvC31[tLast]
        InvB1[tLast:t] = InvB1[tLast]
        
        InvF[t:end] += max(DemandB1[demandMonth]* Ffactor, cycleB1 * Ffactor * 7)
        InvG[t:end] += max(DemandB1[demandMonth]* Gfactor, cycleB1 * Gfactor * 7)
        InvB11[t:end] += max(DemandB1[demandMonth]* B11factor, cycleB1 * B11factor * 7)
        InvC31[t:end] += max(DemandB1[demandMonth]* C31factor, cycleB1 * C31factor * 7)
    else
        if InvB1[t] > DemandB1[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end
        while ((InvF[t]-Ffactor)>0) && ((InvC31[t]-C31factor)>0) && ((InvG[t]-Gfactor)>0) && ((InvB11[t]-B11factor)>0) && ((t+1)<= days) && keepProduction          
            
            prod = round(Int64, 200*randn(1)[1] + cycleB1/2)
            nF = div(InvF[t], Ffactor)
            nG = div(InvG[t], Gfactor)
            nB11 = div(InvB11[t], B11factor)
            nC31 = div(InvC31[t], C31factor)
            
            nB1 = min(nF, nG, nB11, nC31, prod)
            
            InvB1[t:end] += nB1
            InvF[t:end] -= nB1*Ffactor
            InvG[t:end] -= nB1*Gfactor
            InvB11[t:end] -= nB1*B11factor    
            InvC31[t:end] -= nB1*C31factor 
            defect = rand()/10
            
            InvB1[t:end] -= round(Int64, InvB1[t+1]*defect)
            if InvB1[t] > DemandB1[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvB1[t:end] -= DemandB1[demandMonth]
                demandMonth += 1
            end
            Fleft = (DemandB1[demandMonth] - InvB1[t]) * Ffactor
            Gleft = (DemandB1[demandMonth] - InvB1[t]) * Gfactor
            B11left = (DemandB1[demandMonth] - InvB1[t]) * B11factor
            C31left = (DemandB1[demandMonth] - InvB1[t]) * C31factor
            if ((InvF[t] < Fleft) || (InvG[t] < Gleft) || (InvB11[t] < B11left)|| (InvC31[t] < C31left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvF[tLast:tOrder] = InvF[tLast]
                InvG[tLast:tOrder] = InvG[tLast]
                InvB11[tLast:tOrder] = InvB11[tLast]
                InvC31[tLast:tOrder] = InvC31[tLast]
                InvB1[tLast:tOrder] = InvB1[tLast]

                #InvF[tOrder:end] += max(Fleft* Ffactor, cycleB1 * Ffactor * 7)
                #InvG[tOrder:end] += max(Gleft* Gfactor, cycleB1 * Gfactor * 7)
                #InvB11[tOrder:end] += max(B11left* B11factor, cycleB1 * B11factor * 7)
                #InvC31[tOrder:end] += max(C31left* C31factor, cycleB1 * C31factor * 7)
            end
            if !noOrder
                if !(((InvF[t]-Ffactor)>0) && ((InvC31[t]-C31factor)>0) && ((InvG[t]-Gfactor)>0) && ((InvB11[t]-B11factor)>0) && ((t+1)<= days) && keepProduction)
                    noOrder = true
                    t = tOrder
                    InvF[tOrder:end] += max(Fleft* Ffactor, cycleB1 * Ffactor * 7)
                    InvG[tOrder:end] += max(Gleft* Gfactor, cycleB1 * Gfactor * 7)
                    InvB11[tOrder:end] += max(B11left* B11factor, cycleB1 * B11factor * 7)
                    InvC31[tOrder:end] += max(C31left* C31factor, cycleB1 * C31factor * 7)
                end
            end
            
        end
    end
    t+=1
end




In [7]:
using Plots
plot(1:(days+50), InvF/Ffactor, label="F scaled", title="Inventory over 5 months")
plot!(1:(days+50), InvG/Gfactor, label = "G scaled")
plot!(1:(days+50), InvC31/C31factor, label="C31 scaled")
plot!(1:(days+50), InvB11/B11factor, label="B11 scaled")
plot!(1:(days+50), InvB1, label="B1 actual")
using Images
using FileIO
savefig("b1invall.png")

In [8]:
plot(1:(days+50), InvB1, label="B1", title="Inventory of B1")
using Images
using FileIO
savefig("b1inv.png")